# Model class

In [1]:
import torch
import torch.nn as nn

In [6]:
class StocksPredictionModel(nn.Module):

    '''
        Let's break all the parameters down:
            num_classes - how many features will our model have
            hidden size - how many features in the hidden state (in our case - always 1, because we are looking for closing price, which is a 1 column otput (1 feature))
            num_layers - how deep our model is. Because we are using only 1 LSTM object, it will be always 1 
    '''
    device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
    )
    def __init__(self, num_classes, hidden_size, num_layers, batch_first = True, input_size = 16):
        
        super().__init__()
        self.kwargs = {'num_classes': num_classes, 
                       'input_size':input_size, 
                       'hidden_size' : hidden_size, 
                       'num_layers' : num_layers, 
                       'batch_first':batch_first}
        self.input_size = input_size
        torch.manual_seed(42)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.custom_ltsm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = batch_first)
        # self.linear_sequence = nn.Sequential(
        #     #nn.ReLU(),
        #     nn.Linear(hidden_size, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 32),
        #     nn.ReLU(),
        #     nn.Linear(32, num_classes))
        self.linear_sequence = nn.Sequential(
            
            
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes))

    # Basically forward function is what we should do with data
    def forward(self, x):
        x = x.to(device)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        h0 = h0.to(device)
        
        c0 = c0.to(device)
        output, (hn, cn) = self.custom_ltsm(x, (h0, c0))
        hn = hn.view(-1, self.hidden_size)
        hn = hn.to(device)
        
        return self.linear_sequence(hn)
       